### https://docs.julialang.org/en/v1/manual/metaprogramming/

---
**Examples of using *macro*s**

In [ ]:
@time [log(exp(sin(cos(tan(phi))))) for phi in 1:1000];

In [ ]:
using BenchmarkTools
@btime [log(exp(sin(cos(tan(phi))))) for phi in 1:1000];

In [ ]:
@assert sin(π) == 0.0  # notice that our code is cited in the message

In [ ]:
@macroexpand @assert 1 == 0

In [ ]:
@which 1 + 1.0

In [ ]:
@which(1 + 1.0)  # equivalent syntax; no space before the first parenthesis

In [ ]:
@code_lowered sin(2)

See also `@code_typed`, `@code_llvm `, `@code_native`, `@inline`, `@noinline`, `@inbounds`, `@benchmark`, `Base.@pure`

---
***Expression*s**

In [ ]:
x = 2
my_string = "1 + $x"
println(my_string)

In [ ]:
my_code = :(1 + $x)

In [ ]:
typeof(my_code)

In [ ]:
Meta.parse(my_string) == my_code

In [ ]:
?Expr

In [ ]:
my_code.head

In [ ]:
my_code.args

In [ ]:
my_code_again = quote
    1 + $x
end

In [ ]:
my_code_yet_again = Expr(:call, :+, 1, :($x))

In [ ]:
my_code == my_code_yet_again

In [ ]:
eval(my_code)

In [ ]:
dump(my_code)

---
***Symbol*s**

In [ ]:
my_symbol = :x

In [ ]:
typeof(my_symbol)

In [ ]:
?Symbol

In [ ]:
my_new_symbol = Symbol("new_", my_symbol)

In [ ]:
Char(120)

In [ ]:
my_new_symbol_again = Symbol("new_york"[1:4], Char(120))

In [ ]:
my_new_symbol == my_new_symbol_again

In [ ]:
typeof(:x), typeof(:"x"), typeof(:($x)), typeof(:(1)), :1 == 1

---
***Expression*s with *symbol*s**

In [ ]:
my_other_code = :(1 + x)

In [ ]:
my_other_code.args

In [ ]:
my_other_code_again = Expr(:call, :+, 1, Symbol("x"))

In [ ]:
my_other_code == my_other_code_again

In [ ]:
now_with_a_subscript = Expr(:call, :+, 1, Symbol("x_", 1+2))

In [ ]:
x_3 = 100
eval(now_with_a_subscript)

---
**Custom *string literals***

In [ ]:
macro UP_str(string)  # note the `_str` syntax
    uppercase(string)
end

In [ ]:
UP"hello world!"

In [ ]:
# encode secret messages by shifting the letters by 7 steps

function rotate(steps::Int, char::Char)
    char_0 = islowercase(char) ? 'a' : isuppercase(char) ? 'A' : return char
    (char - char_0 + steps) % 26 + char_0
end

println(rotate(7, 'A'))

rotate(steps::Int, string::AbstractString) = map(char -> rotate(steps, char), string)

println(rotate(7, "ABC"))

macro R7_str(string) rotate(7, string) end

R7"ABC"

---
***Macro*s**

*A macro maps a tuple of arguments to a returned expression, and the resulting expression is compiled directly rather than requiring a runtime eval call.*

In [ ]:
macro add_ń(argument)
    :(ń + $argument)
end

In [ ]:
macro add_ź(argument)
    esc(:(ź + $argument))
end

In [ ]:
@macroexpand @add_ń 2

In [ ]:
@macroexpand @add_ź 2

In [ ]:
ń = 1
function do_something()
    ź = 10
    ń = 100
    println(@add_ź 2)
    println(@add_ń 2)
end

do_something()

---
**Some stuff that is good to know.**

slurping

In [ ]:
my_slurp(x...) = x
my_slurp("abc", 1, 'a', 2.)

In [ ]:
typeof(my_slurp("abc", 1, 'a', 2.))

In [ ]:
function my_slurp(x::Vararg{T, N}) where {T, N}
    N == 2 ? "I hate pairs of $T" : x
end

In [ ]:
methods(my_slurp)

In [ ]:
my_slurp("abc", 1, 'a', 2.)

In [ ]:
my_slurp(0, 1.0)

In [ ]:
my_slurp(0, 1)

splatting

In [ ]:
x = (1,2)

In [ ]:
x...

In [ ]:
println(x)

In [ ]:
println(x...)

In [ ]:
+(x...)

In [ ]:
my_tuple = (1,2,3)

head(t::Tuple) = t[1]
_tail(_, t...) = t
tail(t::Tuple) = _tail(t...)

println( head(my_tuple) )
println( tail(my_tuple) )

---
**Exercises**

1. Create the `Rn` string encoding similar to the `R7` above for all `n = 1, ..., 25`. (Use the `rotate` function defined above).

In [ ]:
for n = 1:25
    #
    # ...
    #
end

In [ ]:
# R13"Njrfbzr"

2. Write a macro that accepts an Integer `n` and a function `f` as arguments, and creates a function that is the `f` nested `n`-times, that is to say `x -> f(f(...f(x)...))` `n`-times.

In [ ]:
macro nest(n::Int, f)
    #
    # ...
    #
end

In [ ]:
# @assert (@nest 4 sin)(123) == sin(sin(sin(sin(123))))

3. Write `my_reduce` function that takes a binary operator `*` (`x, y -> *(x,y) = x * y`) and a tuple `t = (t1, t2, ..., tn)` as arguments, and returns `(...((t1 * t2) * t3)... * tn)`. Try to use splatting and slurping. 

In [ ]:
# @assert my_reduce(*, (1,2,3,4)) == reduce(*, (1,2,3,4))

---
Some proposed solutions are below

---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---

1. 

In [ ]:
for n = 1:25
    eval(quote
            macro $(Symbol(:R, n, :_str))(string) rotate($n, string) end
         end
    )
end

2. 

In [ ]:
macro nest(n::Int, f) 
    expression = :x
    for _ = 1 : n
       expression = Expr(:call, f, expression)
    end
    :(x -> $expression)
end

3.

In [ ]:
my_reduce(f, tuple) = _reduce(f, tuple...)
_reduce(f, x, y) = f(x, y)
_reduce(f, x, y, z...) = _reduce(f, f(x, y), z...)